In [5]:
from agents.rag_agent import get_agent


In [3]:
import pandas as pd
df = pd.read_csv("/Users/admin/Working/nckh-chatbot-neo4j/chatbot_api/src/teset_fit_0040.csv")

questions = df['user_input']
ground_truth = df['reference']
retrieval_context = df['reference_contexts']
questions

0     TS. Đặng Trọng Hợp đã phát biểu gì trong buổi ...
1          Khoa có gì đặc biệt cho sinh viên mới không?
2     Nguyễn Đức Huy đã nghiên cứu những kỹ thuật gì...
3     ThS. Nguyễn Thanh Hải đã tham gia vào những dự...
4     Nguyễn Văn Hưng đã thực hiện đề tài nào trong ...
5          Nguyễn Đức Thắng thắng giải gì trong OLP’24?
6     TS. Nguyễn Văn Thiện đã phát biểu gì về kỳ thi...
7            Singapore có liên quan gì đến kỳ thi ICPC?
8     Nguyễn Đức Huy và Nguyễn Anh Đức đã tham gia v...
9     Nguyễn Văn Hưng đã nghiên cứu đề tài gì trong ...
10    Nguyễn Anh Đức đã tham gia vào đề tài nào tron...
11    Nguyễn Anh Đức có tham gia đề tài nào trong ng...
12    Nguyễn Anh Tuấn và Nguyễn Anh Đức đã tham gia ...
13    Trong các đề tài nghiên cứu khoa học năm 2021,...
14    Nguyễn Đức Huy đã nghiên cứu về các kỹ thuật t...
15    Nguyễn Đức Huy và Nguyễn Đức Thắng đã tham gia...
Name: user_input, dtype: object

In [6]:
from chains.semantic_search_chunk_chain import get_chunk_retriever
import uuid
openai_retriever = get_chunk_retriever(name='openai')

In [7]:
import time

answers = []
contexts = []
retriever_times = []
invoke_times = []

for query in questions:
    # Measure time for retriever
    start_retriever = time.time()
    retrieved_docs = openai_retriever.invoke(query)
    retriever_time = time.time() - start_retriever
    retriever_times.append(retriever_time)
    
    # Collect context from retriever
    contexts.append([docs.page_content for docs in retrieved_docs])
    
    # Measure time for chat_agent.invoke
    start_invoke = time.time()
    answer = get_agent().invoke(
        {"input": query},
        {"configurable": {"session_id": 1}}
    ).get("output", "No output text provided.")
    invoke_time = time.time() - start_invoke
    invoke_times.append(invoke_time)
    
    # Collect answer
    answers.append(answer)

# Print time statistics



> Entering new AgentExecutor chain...

Invoking: `get_chunk_tool` with `{'question': 'TS. Đặng Trọng Hợp phát biểu gì trong buổi gặp mặt tân sinh viên?'}`


[Document(metadata={'link': ['https://www.haui.edu.vn/vn/su-kien/hoi-nghi-lop-truong-bi-thu-chi-doan-nam-hoc-2020-2021-giai-dap-hon-80-y-kien-cua-sinh-vien/63312'], 'filename': ['Hội nghị Lớp trưởng, Bí thư Chi đoàn năm học 2020 - 2021: Giải đáp hơn 80 ý kiến của sinh viên']}, page_content='content: như trong cuộc sống. Tại Hội nghị, PGS.TS. Phạm Văn Bổng - Phó Hiệu trưởng Nhà trường đã trực tiếp giải đáp hơn 80 ý kiến từ sinh viên gửi về, về cơ bản, các em đánh giá tốt sự hỗ trợ của nhà trường, thầy cô trong thời gian dịch bệnh, các em cũng đề nghị thầy Phó Hiệu trưởng làm rõ thêm một số vấn đề như: Thẻ Bảo hiểm Y tế, học online, về thực tập, học quân sự, về học ngoại ngữ… TS. Hoàng Thị Thanh Huyền- Phó trưởng phòng Công tác Sinh viên báo cáo tại Hội nghị Kết thúc Hội nghị, PGS.TS. Phạm Văn Bổng gửi lời chúc mừng năm mới đến toà

In [8]:
import datetime

# Get the current date and time
current_datetime = datetime.datetime.now()

# Format the current date and time
formatted_datetime = current_datetime.strftime("%y%m%d-%H%M")

# Create a dynamic filename
filename = f"response_fit{formatted_datetime}"

print(filename)

df["response"] = answers
df["retrieved_contexts"] = contexts
df["retriever_time"] = retriever_times
df["invoke_time"] = invoke_times
df.to_csv(f"{filename}.csv")

response_fit241222-0046


In [14]:
# from ragas.dataset_schema import EvaluationDataset
# from datasets import Dataset
# import pandas as pd
# import ast
# from llm.get_llm import get_eval_model_function

# # Read the CSV file
# df = pd.read_csv(f"{filename}.csv")

# # Define a safe evaluation function
# def safe_literal_eval(val):
#     try:
#         # Try to evaluate the value as a Python literal
#         return ast.literal_eval(val)
#     except (ValueError, SyntaxError):
#         # Return an empty list if evaluation fails
#         return []

# # Apply the safe evaluation function to the columns
# df['retrieved_contexts'] = df['retrieved_contexts'].astype(str).apply(safe_literal_eval)
# df['reference_contexts'] = df['reference_contexts'].astype(str).apply(safe_literal_eval)

# # Process the DataFrame in chunks of 30 rows
# chunk_size = 5

# for start_idx in range(0, len(df), chunk_size):
#     # Slice the DataFrame to get the current chunk
#     df_chunk = df.iloc[start_idx:start_idx + chunk_size]
    
#     # Convert the chunk to a Hugging Face Dataset
#     dataset = Dataset.from_pandas(df_chunk)

#     from ragas import evaluate
#     from ragas.cost import get_token_usage_for_openai
#     from ragas.metrics import (
#         faithfulness,
#         answer_relevancy,
#         context_recall,
#         context_precision,
#         answer_similarity,
#     )

#     # Evaluate the dataset
#     result = evaluate(
#         dataset=dataset,
#         llm=get_eval_model_function(),
#         metrics=[
#             context_precision,
#             context_recall,
#             faithfulness,
#             answer_relevancy,
#             # answer_similarity,
#         ],
#         token_usage_parser=get_token_usage_for_openai,
        
#     )
#     print(result.total_tokens())


#     # Convert results to a DataFrame
#     df_result = result.to_pandas()
    
#     # Add extra columns (adjust indexing for time arrays if necessary)
#     df_result["retriever_time"] = retriever_times[start_idx:start_idx + chunk_size]
#     df_result["invoke_time"] = invoke_times[start_idx:start_idx + chunk_size]

#     # Append results to the output file
#     df_result.to_csv(
#         f"result_{filename}1111.csv",
#         mode='a',  # Append mode
#         header=(start_idx == 0),  # Write the header only for the first chunk
#         index=False
#     )

get llm 


Evaluating: 100%|██████████| 20/20 [02:58<00:00,  8.91s/it]


input_tokens=100297 output_tokens=7953 model=''
get llm 


Evaluating:  85%|████████▌ | 17/20 [03:00<00:44, 14.70s/it]Exception raised in Job[6]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Evaluating: 100%|██████████| 20/20 [03:00<00:00,  9.01s/it]


KeyError: 0

In [10]:
print(result.total_tokens())


input_tokens=116979 output_tokens=7306 model=''


In [61]:
# df_result["retriever_time"] = retriever_times[30:]
# df_result["invoke_time"] = invoke_times[30:]
# df_result.to_csv(f"second_result_{filename}")